In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [28]:
class FastBertPredictor:
    def __init__(self, model_path='Best_82'):
        # Load model once during initialization
        self.model = tf.keras.models.load_model(
            model_path,
            custom_objects={'KerasLayer': hub.KerasLayer}
        )
        # Compile the model
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
    def predict(self, text):
        # Convert input to array if it's a single string
        if isinstance(text, str):
            text = np.array([text])
        elif isinstance(text, list):
            text = np.array(text)
            
        # Make prediction
        prediction = self.model.predict(text, verbose=0)  # Set verbose=0 to suppress output
        print(prediction)
        return prediction

# Initialize predictor once (globally)
predictor = FastBertPredictor()

def quick_predict(text):
    """
    Fast prediction for a single text
    """
    prediction = predictor.predict(text)
    return "positive" if prediction[0] == 1 else "negative"

In [ ]:
# Example usage
if __name__ == "__main__":
    # Single text prediction
    text = "I am Nishchay, I love to play badminton, I hate the taste of bittergord, but I like swimming"
    result = quick_predict(text)
    print(f"Text: '{text}'\nSentiment: {result}")
    
    # Multiple texts prediction (if needed)
    texts = [
        "I am Nishchay, I love to play badminton, I hate the taste of bittergord, but I like swimming"
    ]
    
    predictions = predictor.predict(texts)
    for text, pred in zip(texts, predictions):
        sentiment = "positive" if pred == 1 else "negative"
        print(f"\nText: '{text}'\nSentiment: {sentiment}")

[[0.67554706]]
Text: 'I am Nishchay, I love to play badminton, I hate the taste of bittergord, but I like swimming'
Sentiment: negative
[[0.67554706]]

Text: 'I am Nishchay, I love to play badminton, I hate the taste of bittergord, but I like swimming'
Sentiment: negative


In [73]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [74]:
wml_creds ={
    "apikey": '_KnZKtXJvhFYIq2z5Ot08WKKpfBwbsgyj3CWsxYHJyds',
    "url": "https://us-south.ml.cloud.ibm.com"
}

In [75]:
wml_client = APIClient(wml_credentials=wml_creds)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----------------------  ------------------------
ID                                    NAME                    CREATED
6760b3b6-140d-46c0-861e-2eebda2dd6ab  FEMA-SentimentAnalysis  2024-11-11T16:08:16.474Z
------------------------------------  ----------------------  ------------------------


,ID,NAME,CREATED
0,6760b3b6-140d-46c0-861e-2eebda2dd6ab,FEMA-SentimentAnalysis,2024-11-11T16:08:16.474Z


In [76]:
SPACE_ID ='6760b3b6-140d-46c0-861e-2eebda2dd6ab'

In [77]:
wml_client.set.default_space(SPACE_ID)
print(f"Default space is set to: {SPACE_ID}")

Default space is set to: 6760b3b6-140d-46c0-861e-2eebda2dd6ab


In [78]:
# Set Python Version and Runtime for TensorFlow
software_spec_uid = wml_client.software_specifications.get_id_by_name('tensorflow_rt23.1-py3.10')

# # Setup model meta for custom TensorFlow model
# model_props = {
#     wml_client.repository.ModelMetaNames.NAME: 'FastBert Model', 
#     wml_client.repository.ModelMetaNames.TYPE: 'tensorflow_2.12',
#     wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
# }
model_dir = 'model_package/Best_82/saved_model.pb'

# Define the location of requirements.txt
requirements_file = 'requirements.txt'

# Define the location of pipeline.py
pipeline_file = 'pipeline.py'

# Create the client to interact with WML

# Upload the model
model_metadata = {
    wml_client.repository.ModelMetaNames.NAME: "Best_82",
    wml_client.repository.ModelMetaNames.TYPE: "tensorflow_2.12",
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

# Store the model
model_uid = wml_client.repository.store_model(model = model_dir, meta_props=model_metadata)

# Print model UID
print(f"Model uploaded with UID: {model_uid}")

WMLClientError: Saving trained model in repository failed. 'model_package/Best_82/saved_model.pb' file does not have valid format

In [63]:
# model_details = wml_client.repository.store_model(
#     model='model_package',  # Path to the zip file
#     meta_props=model_props
# )
# model_details

model_uid = model_uid['metadata']['id']

deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: 'FEMA-SentimentAnalysis',
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {
        "dependencies": {
            "pip_requirements": requirements_file,  # Point to the requirements file
            "python": "3.10"  # Ensure the Python version is appropriate
        },
        "pipeline": pipeline_file  # Ensure the pipeline file is used for inference
    }
}

# Deploy the model
deployment = wml_client.deployments.create(
    artifact_uid=model_uid,
    meta_props=deployment_props
)

# Output result
print(f"Deployment created: {deployment}")





#######################################################################################

Synchronous deployment creation for uid: '882a737a-a1a2-47a7-be8c-1113bdabc7ce' started

#######################################################################################


initializing
Note: Software specifications Tensorflow 2.12 is deprecated. Use Tensorflow with a supported model type and software specification. For details, see https://dataplatform.cloud.ibm.com/docs/content/wsj/wmls/wmls-deploy-python-types.html?context=cpdaas.
.......

Deployment creation failed for deployment id: fb53b529-17ac-45e2-b65b-687bc6944e45. Errors: [{'code': 'f63377faa6becd4a47dcc7067ba1c40d', 'message': "Tensorflow model load failed with error: Op type not registered 'CaseFoldUTF8' in binary running on wml-dep-od-tensorflow2.12-cl-yn19y5di-7f45c899bc-sx4cj. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.."}]



failed

--------------------------
Deployment creation failed
--------------------------


Tensorflow model load failed with error: Op type not registered 'CaseFoldUTF8' in binary running on wml-dep-od-tensorflow2.12-cl-yn19y5di-7f45c899bc-sx4cj. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed..


WMLClientError: Deployment creation failed for deployment id: fb53b529-17ac-45e2-b65b-687bc6944e45. Errors: [{'code': 'f63377faa6becd4a47dcc7067ba1c40d', 'message': "Tensorflow model load failed with error: Op type not registered 'CaseFoldUTF8' in binary running on wml-dep-od-tensorflow2.12-cl-yn19y5di-7f45c899bc-sx4cj. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.."}]

In [41]:
model_uid = wml_client.repository.get_model_id(model_details)
print(f'Model UID: {model_uid}')

Model UID: 43bbd541-907f-4594-b0ed-afff335f7a49


In [ ]:
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: 'FEMA-SentimentAnalysis', 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# Deploy the model
deployment = wml_client.deployments.create(
    artifact_uid=model_uid, 
    meta_props=deployment_props
)

# Output result
print(deployment)


In [ ]:
software_specs = wml_client.software_specifications.list()
for spec in software_specs['resources']:
    print(spec['name'])

In [79]:
import tensorflow as tf
import tensorflow_hub as hub

class FastBertPredictor:
    def __init__(self, model_path='Best_82'):
        # Load the model with custom objects
        self.model = tf.keras.models.load_model(
            model_path,
            custom_objects={'KerasLayer': hub.KerasLayer}
        )

    def save_model(self, save_path='Best_82_saved'):
        # Re-save the model in TensorFlow's SavedModel format explicitly
        tf.saved_model.save(self.model, save_path)
        print(f"Model saved in TensorFlow SavedModel format at {save_path}")

# Initialize predictor and re-save the model
predictor = FastBertPredictor('Best_82')
predictor.save_model('Best_82_saved')


RuntimeError: Op type not registered 'CaseFoldUTF8' in binary running on HONC02FT05FML7M. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.